<a href="https://colab.research.google.com/github/rjainn/lb-p1/blob/main/Labelbox_Intern_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

In [ ]:
pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 957.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
import pandas as pd
import uuid

In [ ]:
train_df = pd.DataFrame(dataset['train'].to_pandas())
test_df = pd.DataFrame(dataset['test'].to_pandas())
validation_df = pd.DataFrame(dataset['validation'].to_pandas())

In [ ]:
df = pd.concat([train_df, test_df, validation_df], ignore_index=True)
df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
16364,13829423,Carla: I've got it...\r\nDiego: what?\r\nCarla...,Carla's date for graduation is on June 4th. Di...
16365,13727710,"Gita: Hello, this is Beti's Mum Gita, I wanted...",Bev is going on the school trip with her son. ...
16366,13829261,"Julia: Greg just texted me\r\nRobert: ugh, del...",Greg cheated on Julia. He apologises to her. R...
16367,13680226,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n...",Marry broke her nail and has a party tomorrow....


In [ ]:
df['dialogue'] = df['dialogue'].str.split(r'[\r\n]+')

In [ ]:
df.head()


,id,dialogue,summary
0,13818513,"[Amanda: I baked cookies. Do you want some?, ...",Amanda baked cookies and will bring Jerry some...
1,13728867,[Olivia: Who are you voting for in this electi...,Olivia and Olivier are voting for liberals in ...
2,13681000,"[Tim: Hi, what's up?, Kim: Bad mood tbh, I was...",Kim may try the pomodoro technique recommended...
3,13730747,"[Edward: Rachel, I think I'm in ove with Bella...",Edward thinks he is in love with Bella. Rachel...
4,13728094,"[Sam: hey overheard rick say something, Sam: ...","Sam is confused, because he overheard Rick com..."


In [ ]:
import time

In [ ]:
def transform_to_labelbox_conversation_format(dataset):
    conversations = dataset['dialogue']
    labelbox_entries = []
    conversations_array = []
    for conversation in conversations:
      message_id = 1
      messages_array = []
      people_ids = {}
      for message in conversation:
        message_split = message.split(':')
        if (len(message_split) >= 2):
          sender = str(message_split[0])
          message_content = str(message_split[1])
        if sender not in people_ids:
          random_id = str(uuid.uuid4())
          people_ids[sender] = random_id

        labelbox_message = {
            "messageId" : f"message-{message_id}",
            "timestampUsec" : int(time.time() * 1e6),
            "content" : message_content,
            "user" : {
                "userId" : people_ids[sender],
                "name" : sender.capitalize()
            },
            "align" : "left" if message_id % 2 == 0 else "right",
            "canLabel" : True
        }
        message_id += 1
        messages_array.append(labelbox_message)
      labelbox_conversation = {
          "type": 'application/vnd.labelbox.conversational',
          "version" : 1,
          "messages" : messages_array
      }
      labelbox_entries.append(labelbox_conversation)
    return labelbox_entries

In [ ]:
labelbox_conversations = transform_to_labelbox_conversation_format(df)

In [ ]:
aws_access_key = 'ACCESS_KEY_HERE'
aws_secret_access = 'SECRET_ACCESS_KEY_HERE'
s3_folder_path = 'transformed_conversations/'

In [ ]:
import json
import boto3
import os

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=aws_access_key ,
                  aws_secret_access_key=aws_secret_access)

# Assuming your S3 bucket name is 'your-s3-bucket'
bucket_name = 'conversations-data-transformed-bucket'

In [ ]:
for index, conversation in enumerate(labelbox_conversations):

    with open(f'conversation_{index + 1}.json', 'w') as json_file:
        json.dump(conversation, json_file)

    file_name = f'conversation_{index + 1}.json'

    s3.upload_file(file_name, bucket_name, f'{s3_folder_path}/{file_name}')

    os.remove(file_name)

In [ ]:
!pip install labelbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.7 MB/s eta 0:00:00


In [ ]:
import labelbox as lb
import boto3

In [ ]:
labelbox_api_key = 'API_KEY_HERE'

In [ ]:
from labelbox import Project, Dataset, Client

In [ ]:
client = lb.Client(api_key=labelbox_api_key)

In [ ]:
project = client.create_project(
    name = "Labelbox Project #1",
    description = "Importing a huggingface dataset using conversational datatype, homework#1",
    media_type = lb.MediaType.Conversational
)

In [ ]:
import uuid

In [ ]:
print(len(labelbox_conversations))

16369


In [ ]:
dataset = client.create_dataset(name="Bulk import example - Conversational")

uploads = []
for i in range(1,len(labelbox_conversations) + 1):
  uploads.append({
      "row_data" : f"https://conversations-data-transformed-bucket.s3.us-east-1.amazonaws.com/transformed_conversations//conversation_{i}.json",
      "global_key" : "TEST-ID-%id" % uuid.uuid1(),
      "media_type" : "CONVERSATIONAL"
  })

task = dataset.create_data_rows(uploads)
task.wait_till_done()
print(task.errors)

None
